In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install pandas scikit-learn transformers sentencepiece
!pip install googletrans==4.0.0-rc1

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 26.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 2.3 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-4.0.0rc1-py3-none-any.whl size=17396 sha256=56f752de66f76dcfc4dd1f89f57d096aa039da7c3c5f1d7212af3df079680d2f
  Stored in directory: /root/.cache/pip/wheels/95/0f/04/b17a72024b56a60e499ce1a6313d283ed5ba332407155bee03
Successfully built googletrans
  Attempting uninstall: hyperframe
    Found existing installation: hyperframe 6.1.0
    Uninstalling hyperfram

In [ ]:
# Load datasets
import pandas as pd
idn = pd.read_csv("/content/drive/MyDrive/deteksi_cd-indo/main-indonesia.csv")
kgl = pd.read_csv("/content/drive/MyDrive/deteksi_cd-indo/kaggle.csv")

In [4]:
# Tampilkan datasets
display(idn)

,TEXT,DATA STATUS,FIRST ANNOTATOR,SECOND ANNOTATOR
0,ketika saya sudah berusaha semaksimal mungkin ...,RAW-ORI,All-or-nothing,All-or-nothing
1,Saya ketika SD hingga SMP dulu beranggapan bah...,RAW-ORI,All-or-nothing,All-or-nothing
2,Saya berfikir jika tidak lolos MBKM maka masa ...,RAW-ORI,All-or-nothing,All-or-nothing
3,"Di desa saya dulu ada dua kelompok pertemanan,...",RAW-ORI,All-or-nothing,All-or-nothing
4,Saya hanya berteman dengan orang yang tidak me...,RAW-ORI,All-or-nothing,All-or-nothing
...,...,...,...,...
4987,Suatu saat aku merasa kalau dia mencintaiku da...,DIS-TG,Emotional Reasoning,Emotional Reasoning
4988,Saya percaya bahwa restoran lokal adalah tempa...,DIS-TG,Emotional Reasoning,Emotional Reasoning
4989,Selang beberapa hari terbukti dia selingkuh.,DIS-TG,Emotional Reasoning,Emotional Reasoning
4990,"Faktanya, mereka berbohong kepada saya.",DIS-TG,Emotional Reasoning,Emotional Reasoning


In [9]:
idn = idn.drop(columns=['DATA STATUS', 'SECOND ANNOTATOR']).rename(columns={'TEXT':'text', 'FIRST ANNOTATOR':'label'})
display(idn)

,text,label
0,ketika saya sudah berusaha semaksimal mungkin ...,All-or-nothing
1,Saya ketika SD hingga SMP dulu beranggapan bah...,All-or-nothing
2,Saya berfikir jika tidak lolos MBKM maka masa ...,All-or-nothing
3,"Di desa saya dulu ada dua kelompok pertemanan,...",All-or-nothing
4,Saya hanya berteman dengan orang yang tidak me...,All-or-nothing
...,...,...
4987,Suatu saat aku merasa kalau dia mencintaiku da...,Emotional Reasoning
4988,Saya percaya bahwa restoran lokal adalah tempa...,Emotional Reasoning
4989,Selang beberapa hari terbukti dia selingkuh.,Emotional Reasoning
4990,"Faktanya, mereka berbohong kepada saya.",Emotional Reasoning


In [10]:
idn['label'].unique()

array(['All-or-nothing', 'Discounting the positives',
       'Emotional Reasoning', 'Fortune-telling', 'Labeling',
       'Magnification or Minimization', 'Mental filter', 'Mind Reading',
       'Overgeneralization', 'Personalization and Blame',
       'Should statement', 'No Distortion'], dtype=object)

In [11]:
idn['label'] = idn['label'].replace({
    'Fortune-telling':'Jumping to Conclusions',
    'Mind Reading':'Jumping to Conclusions'
})
idn.head()

,text,label
0,ketika saya sudah berusaha semaksimal mungkin ...,All-or-nothing
1,Saya ketika SD hingga SMP dulu beranggapan bah...,All-or-nothing
2,Saya berfikir jika tidak lolos MBKM maka masa ...,All-or-nothing
3,"Di desa saya dulu ada dua kelompok pertemanan,...",All-or-nothing
4,Saya hanya berteman dengan orang yang tidak me...,All-or-nothing


In [6]:
display(kgl)

,Id_Number,Patient Question,Distorted part,Dominant Distortion,Secondary Distortion (Optional)
0,4500,"Hello, I have a beautiful,smart,outgoing and a...",The voice are always fimilar (someone she know...,Personalization,NaN
1,4501,Since I was about 16 years old I’ve had these ...,I feel trapped inside my disgusting self and l...,Labeling,Emotional Reasoning
2,4502,So I’ve been dating on and off this guy for a...,NaN,No Distortion,NaN
3,4503,My parents got divorced in 2004. My mother has...,NaN,No Distortion,NaN
4,4504,I don’t really know how to explain the situati...,I refused to go because I didn’t know if it wa...,Fortune-telling,Emotional Reasoning
...,...,...,...,...,...
2525,2562,I’m a 21 year old female. I spent most of my l...,NaN,No Distortion,NaN
2526,2563,I am 21 female and have not had any friends fo...,Now I am at university my peers around me all ...,Overgeneralization,NaN
2527,2564,From the U.S.: My brother is 19 years old and ...,He claims he’s severely depressed and has outb...,Mental filter,Mind Reading
2528,2565,From the U.S.: I am a 21 year old woman who ha...,NaN,No Distortion,NaN


In [7]:
kgl['Dominant Distortion'].unique()

array(['Personalization', 'Labeling', 'No Distortion', 'Fortune-telling',
       'Magnification', 'Mind Reading', 'All-or-nothing thinking',
       'Overgeneralization', 'Mental filter', 'Emotional Reasoning',
       'Should statements'], dtype=object)

In [8]:
# Translate otomatis dataset dari kaggle EN-ID
from googletrans import Translator
translator = Translator()

def translate_lightweight(texts):
    result = []
    for t in texts:
        try:
            res = translator.translate(t, src='en', dest='id').text
        except:
            res = t
        result.append(res)
    return result

kgl['Patient Question'] = translate_lightweight(kgl['Patient Question'].tolist())
display(kgl)


,Id_Number,Patient Question,Distorted part,Dominant Distortion,Secondary Distortion (Optional)
0,4500,"Halo, saya memiliki seorang gadis kecil berusi...",The voice are always fimilar (someone she know...,Personalization,NaN
1,4501,"Sejak saya berusia sekitar 16 tahun, saya meng...",I feel trapped inside my disgusting self and l...,Labeling,Emotional Reasoning
2,4502,So I’ve been dating on and off this guy for a...,NaN,No Distortion,NaN
3,4503,Orang tua saya bercerai pada tahun 2004. Ibu s...,NaN,No Distortion,NaN
4,4504,Aku tidak benar-benar tahu bagaimana menjelask...,I refused to go because I didn’t know if it wa...,Fortune-telling,Emotional Reasoning
...,...,...,...,...,...
2525,2562,Saya seorang wanita berusia 21 tahun.Saya meng...,NaN,No Distortion,NaN
2526,2563,Saya perempuan berusia 21 tahun dan belum memp...,Now I am at university my peers around me all ...,Overgeneralization,NaN
2527,2564,Dari AS: Kakak laki-laki saya berusia 19 tahun...,He claims he’s severely depressed and has outb...,Mental filter,Mind Reading
2528,2565,Dari AS: Saya seorang wanita berusia 21 tahun ...,NaN,No Distortion,NaN


In [13]:
kgl = kgl.drop(columns=['Id_Number', 'Distorted part', 'Secondary Distortion (Optional)']).rename(columns={'Patient Question':'text', 'Dominant Distortion':'label'})
kgl.head()

,text,label
0,"Halo, saya memiliki seorang gadis kecil berusi...",Personalization
1,"Sejak saya berusia sekitar 16 tahun, saya meng...",Labeling
2,So I’ve been dating on and off this guy for a...,No Distortion
3,Orang tua saya bercerai pada tahun 2004. Ibu s...,No Distortion
4,Aku tidak benar-benar tahu bagaimana menjelask...,Fortune-telling


In [15]:
kgl['label'] = kgl['label'].replace({
    'Personalization':'Personalization and Blame',
    'Fortune-telling':'Jumping to Conclusions',
    'Magnification':'Magnification or Minimization',
    'Mind Reading':'Jumping to Conclusions',
    'All-or-nothing thinking':'All-or-nothing'
})
kgl.head()

,text,label
0,"Halo, saya memiliki seorang gadis kecil berusi...",Personalization and Blame
1,"Sejak saya berusia sekitar 16 tahun, saya meng...",Labeling
2,So I’ve been dating on and off this guy for a...,No Distortion
3,Orang tua saya bercerai pada tahun 2004. Ibu s...,No Distortion
4,Aku tidak benar-benar tahu bagaimana menjelask...,Jumping to Conclusions


In [17]:
# Gabungkan baris dari kedua dataset
dataset_distorsi_kognitif_idn = pd.concat([idn, kgl], ignore_index=True)

# Buat kolom id (mulai dari 1)
dataset_distorsi_kognitif_idn['id'] = range(1, len(dataset_distorsi_kognitif_idn) + 1)

# Pindahkan kolom 'id' ke posisi paling kiri
cols = ['id'] + [col for col in dataset_distorsi_kognitif_idn.columns if col != 'id']
dataset_distorsi_kognitif_idn = dataset_distorsi_kognitif_idn[cols]

# Cek jumlah data hasil merge
print("Total baris gabungan:", len(dataset_distorsi_kognitif_idn))
dataset_distorsi_kognitif_idn.head()

Total baris gabungan: 7522


,id,text,label
0,1,ketika saya sudah berusaha semaksimal mungkin ...,All-or-nothing
1,2,Saya ketika SD hingga SMP dulu beranggapan bah...,All-or-nothing
2,3,Saya berfikir jika tidak lolos MBKM maka masa ...,All-or-nothing
3,4,"Di desa saya dulu ada dua kelompok pertemanan,...",All-or-nothing
4,5,Saya hanya berteman dengan orang yang tidak me...,All-or-nothing


In [ ]:
output_path = '/content/drive/MyDrive/deteksi_cd-indo/dataset.csv'
dataset_distorsi_kognitif_idn.to_csv(output_path, index=False)
print("✅ File gabungan disimpan ke:", output_path)

✅ File gabungan disimpan ke: /content/drive/MyDrive/Deteksi_Distorsi-Kognitif-IDN/dataset.csv
